In [150]:
# Importar librerias
import datetime
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from scipy.sparse import hstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Embedding, Flatten, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.random import set_seed
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
corpus = pd.read_csv('/content/obligatorio.csv', index_col = 0)
corpus['label'].value_counts()

real    4480
fake    4080
Name: label, dtype: int64

In [5]:
test = pd.read_csv('/content/test_without_label.csv', index_col = 0)

In [6]:
# Return top n (5 by default) rows of a data frame. 
corpus.head()

,tweet,label
id,,
1,Chinese converting to Islam after realising th...,fake
2,11 out of 13 people (from the Diamond Princess...,fake
3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
4,Mike Pence in RNC speech praises Donald Trump’...,fake
5,6/10 Sky's @EdConwaySky explains the latest #C...,real


In [312]:
X = corpus['tweet'] 
Y = corpus['label']

In [313]:
X_test = test["tweet"]

In [314]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, random_state = 1) 

In [ ]:
vectorizer = CountVectorizer()

In [ ]:
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train)
X_val = vectorizer.transform(X_val)

In [ ]:
X_test = vectorizer.transform(X_test)

In [ ]:
tablero = pd.DataFrame(X_train.toarray(), columns=vectorizer.get_feature_names_out())
tablero

**Classifiers**

In [ ]:
# Decision Tree Classifier

tree = DecisionTreeClassifier()

_ = tree.fit(tablero, y_train)  

y_pred_tree = tree.predict(X_val)

y_pred_tree_train = tree.predict(X_train)

In [ ]:
report = classification_report(y_val, y_pred_tree, digits=4)
print(report)

In [ ]:
report = classification_report(y_train, y_pred_tree_train, digits=4)
print(report)

In [ ]:
#Baggin Classifier

bagging_tree = BaggingClassifier(DecisionTreeClassifier())

_ = bagging_tree.fit(tablero, y_train)

y_pred_bagging_tree = bagging_tree.predict(X_val)

y_pred_bagging_train = bagging_tree.predict(X_train)

In [ ]:
report = classification_report(y_val, y_pred_bagging_tree, digits=4)
print(report)

In [ ]:
report = classification_report(y_train, y_pred_bagging_train, digits=4)
print(report)

In [363]:
#Random Forest Classifier

rf = RandomForestClassifier()

_ = rf.fit(tablero, y_train)

y_pred_rf = rf.predict(X_val)

y_pred_rf_train = rf.predict(X_train)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [364]:
report = classification_report(y_val, y_pred_rf, digits=4)
print(report)

              precision    recall  f1-score   support

        fake     0.9478    0.9214    0.9344       827
        real     0.9284    0.9525    0.9403       885

    accuracy                         0.9375      1712
   macro avg     0.9381    0.9370    0.9374      1712
weighted avg     0.9378    0.9375    0.9375      1712



In [ ]:
report = classification_report(y_train, y_pred_rf_train, digits=4)
print(report)

In [ ]:
# Ada Boost Classifier

ada = AdaBoostClassifier(n_estimators=100, learning_rate=0.5)

_ = ada.fit(tablero, y_train)

y_pred_ada = ada.predict(X_val)

y_pred_ada_train = ada.predict(X_train)

In [ ]:
report = classification_report(y_val, y_pred_ada, digits=4)
print(report)

In [ ]:
report = classification_report(y_train, y_pred_ada_train, digits=4)
print(report)

In [ ]:
# Gradient Classifier

gradient = GradientBoostingClassifier()

_ = gradient.fit(X_train, y_train) 

y_pred_gradient = gradient.predict(X_val)

y_pred_gradient_train = gradient.predict(X_train)

In [ ]:
report = classification_report(y_val, y_pred_gradient, digits=4)
print(report)

In [ ]:
report = classification_report(y_train, y_pred_gradient_train, digits=4)
print(report)

**Deep Learning**

In [335]:
set_seed(1)

In [346]:
maxlen = 900

tokenizer = Tokenizer()

tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded_seq = pad_sequences(X_train_sequences, truncating = 'post', padding ='post', maxlen = maxlen)

y_train_dl = np.where(y_train == "real", 1, 0)

In [342]:
maxlen = 900

tokenizer = Tokenizer()

tokenizer.fit_on_texts(X_val)

X_val_sequences = tokenizer.texts_to_sequences(X_val)
X_val_padded_seq = pad_sequences(X_val_sequences, truncating = 'post', padding ='post', maxlen = maxlen)

y_val_dl = np.where(y_val == "real", 1, 0)

In [343]:
model = Sequential()
model.add(Embedding(5000, 8, input_length=maxlen))
model.add(Flatten())
model.add(Dense(4))
model.add(Dropout(0.2)) 
model.add(Dense(2))
model.add(Dropout(0.3))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.summary()

Model: "sequential_45"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_39 (Embedding)    (None, 900, 8)            40000     
                                                                 
 flatten_32 (Flatten)        (None, 7200)              0         
                                                                 
 dense_87 (Dense)            (None, 4)                 28804     
                                                                 
 dropout_51 (Dropout)        (None, 4)                 0         
                                                                 
 dense_88 (Dense)            (None, 2)                 10        
                                                                 
 dropout_52 (Dropout)        (None, 2)                 0         
                                                                 
 activation_65 (Activation)  (None, 2)               

In [344]:
criterion = SGD(learning_rate = 0.01) #Empleamos learning rate
model.compile(optimizer=criterion, loss='binary_crossentropy', metrics=['accuracy']) #Es un problema de clasificación, se utiliza la entropía cruzada y como métrica la accuracy. 

In [345]:
callbacks = [keras.callbacks.TensorBoard(),
             keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

model.fit(X_train_padded_seq, y_train_dl, validation_data=(X_val_padded_seq, y_val_dl), batch_size=20, epochs=100, callbacks=callbacks)

y_pred_model_val = model.predict(X_val_padded_seq)
y_pred_model_train = model.predict(X_train_padded_seq)

Epoch 1/100
343/343 [==============================] - 2s 4ms/step - loss: 0.6830 - accuracy: 0.5483 - val_loss: 0.6770 - val_accuracy: 0.5169
Epoch 2/100
343/343 [==============================] - 1s 3ms/step - loss: 0.6551 - accuracy: 0.6043 - val_loss: 0.6402 - val_accuracy: 0.6863
Epoch 3/100
343/343 [==============================] - 1s 4ms/step - loss: 0.6359 - accuracy: 0.6384 - val_loss: 0.6405 - val_accuracy: 0.7091
Epoch 4/100
343/343 [==============================] - 1s 3ms/step - loss: 0.6309 - accuracy: 0.6343 - val_loss: 0.5697 - val_accuracy: 0.7296
Epoch 5/100
343/343 [==============================] - 1s 3ms/step - loss: 0.6217 - accuracy: 0.6343 - val_loss: 0.6205 - val_accuracy: 0.6618
Epoch 6/100
343/343 [==============================] - 1s 3ms/step - loss: 0.6150 - accuracy: 0.6374 - val_loss: 0.6062 - val_accuracy: 0.7068
Epoch 7/100
343/343 [==============================] - 1s 3ms/step - loss: 0.6107 - accuracy: 0.6383 - val_loss: 0.5628 - val_accuracy: 0.7407

In [347]:
y_pred_model_val = np.where(y_pred_model_val >= 0.5, 1, 0)
y_pred_model_train = np.where(y_pred_model_train >= 0.5, 1, 0)


In [348]:
report = classification_report(y_val_dl, y_pred_model_val, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.8074    0.7606    0.7833       827
           1     0.7878    0.8305    0.8086       885

    accuracy                         0.7967      1712
   macro avg     0.7976    0.7955    0.7959      1712
weighted avg     0.7973    0.7967    0.7964      1712



In [323]:
report = classification_report(y_train_dl, y_pred_model_train, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.9042    0.9056    0.9049      3253
           1     0.9145    0.9132    0.9138      3595

    accuracy                         0.9096      6848
   macro avg     0.9094    0.9094    0.9094      6848
weighted avg     0.9096    0.9096    0.9096      6848



**Descarga del archivo**

In [ ]:
# En este caso la predicción se está realizando con Random Forest Classifier.

y_test_pred = rf.predict(X_test)

In [ ]:
pred_pd = pd.DataFrame(y_test_pred, columns = ['label'], index = range(1, 2141)) 

pred_pd.to_csv('submission_1.csv', index_label = 'id')

**Modelo con Tfldf, Stop Word Removal y Reduce to a Root**

In [349]:
X = corpus['tweet'] 
Y = corpus['label']

In [350]:
X_test = test["tweet"]

In [351]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, random_state = 1) 

In [352]:
vectorizer = TfidfVectorizer(stop_words = nltk.corpus.stopwords.words('english'), max_features = 1500)

In [360]:
use_wordcount = False
use_charcount = True
use_hashtagcount = True
use_atsigncount = True
use_percentsigncount = True
use_linkcount = True
use_lettercount = False

vectorizer.fit(X_train)

X_train = vectorizer.transform(X_train)
X_val = vectorizer.transform(X_val)

if use_wordcount:
  X_train = hstack((X_train,np.array(X_train_wordcount)[:,None]))
  X_val = hstack((X_val,np.array(X_val_wordcount)[:,None]))

if use_charcount:
  X_train = hstack((X_train,np.array(X_train_charcount)[:,None]))
  X_val = hstack((X_val,np.array(X_val_charcount)[:,None]))

if use_hashtagcount:
  X_train = hstack((X_train,np.array(X_train_hashtagcount)[:,None]))
  X_val = hstack((X_val,np.array(X_val_hashtagcount)[:,None]))

if use_atsigncount:
  X_train = hstack((X_train,np.array(X_train_atsigncount)[:,None]))
  X_val = hstack((X_val,np.array(X_val_atsigncount)[:,None]))

if use_percentsigncount:
  X_train = hstack((X_train,np.array(X_train_percentsigncount)[:,None]))
  X_val = hstack((X_val,np.array(X_val_percentsigncount)[:,None]))

if use_linkcount:
  X_train = hstack((X_train,np.array(X_train_linkcount)[:,None]))
  X_val = hstack((X_val,np.array(X_val_linkcount)[:,None]))

if use_lettercount:
  X_train = hstack((X_train,np.array(X_train_lettercount)[:,None]))
  X_val = hstack((X_val,np.array(X_val_lettercount)[:,None]))

In [361]:
X_test = vectorizer.transform(X_test)

if use_wordcount:
  X_test = hstack((X_test,np.array(X_test_wordcount)[:,None]))

if use_charcount:
  X_test = hstack((X_test,np.array(X_test_charcount)[:,None]))

if use_hashtagcount:
  X_test = hstack((X_test,np.array(X_test_hashtagcount)[:,None]))

if use_atsigncount:
  X_test = hstack((X_test,np.array(X_test_atsigncount)[:,None]))

if use_percentsigncount:
  X_test = hstack((X_test,np.array(X_test_percentsigncount)[:,None]))

if use_linkcount:
  X_test = hstack((X_test,np.array(X_test_linkcount)[:,None]))

if use_lettercount:
  X_test = hstack((X_test,np.array(X_test_lettercount)[:,None]))

In [362]:
columns = vectorizer.get_feature_names_out()

if use_wordcount:
  columns = np.concatenate([columns,['word count']])

if use_charcount:
  columns = np.concatenate([columns,['char count']])

if use_hashtagcount:
  columns = np.concatenate([columns,['hashtag count']])

if use_atsigncount:
  columns = np.concatenate([columns,['at sign count']])

if use_percentsigncount:
  columns = np.concatenate([columns,['percent sign count']])

if use_linkcount:
  columns = np.concatenate([columns,['link count']])

if use_lettercount:
  columns = np.concatenate([columns,['letter count']])

tablero = pd.DataFrame(X_train.toarray(), columns=columns)
tablero

,00,000,08,09,10,100,1000,10000,100000,10pm,...,zealand,zero,zinc,zp4vylo0pb,zqrpneofet,char count,hashtag count,at sign count,percent sign count,link count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,224.0,4.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,282.0,2.0,0.0,0.0,2.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,277.0,2.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,169.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,205.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,230.0,0.0,0.0,0.0,2.0
6844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,123.0,0.0,0.0,0.0,0.0
6845,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,243.0,4.0,3.0,0.0,1.0
6846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,256.0,1.0,0.0,0.0,2.0


**Reduce to a Root**

In [ ]:
# Stemmer

from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer(language = 'english')
analyzer = TfidfVectorizer().build_analyzer()

def stemmed_words(doc):
  return (stemmer.stem(w) for w in analyzer(doc))

vectorizer = TfidfVectorizer(analyzer=stemmed_words, stop_words = nltk.corpus.stopwords.words('english'), max_df = 0.95)

In [ ]:
# Lemmatizer

from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

def lemmatized_words(doc):
    return (lemmatizer.lemmatize(w.lower()) for w in doc.split())

vectorizer = TfidfVectorizer(analyzer = lemmatized_words, stop_words = nltk.corpus.stopwords.words('english'), max_df = 0.95)

**Funciones**

In [353]:
# Contar caracteres de un string

countCharacters = (lambda x : len(x))

X_train_charcount = (X_train).apply(countCharacters)
X_val_charcount = (X_val).apply(countCharacters)
X_test_charcount = (X_test).apply(countCharacters)

In [354]:
# Contar palabras de un string

countWords = (lambda x : len(x.split()))

X_train_wordcount = (X_train).apply(countWords)
X_val_wordcount = (X_val).apply(countWords)
X_test_wordcount = (X_test).apply(countWords)

In [355]:
# Contar hashtags de un string

countHashtags = (lambda x: len(re.findall(r"#(\w+)", x)))

X_train_hashtagcount = (X_train).apply(countHashtags)
X_val_hashtagcount = (X_val).apply(countHashtags)
X_test_hashtagcount = (X_test).apply(countHashtags)

In [356]:
# Contar arrobas de un string

countAtSigns = (lambda x: len(re.findall(r"@(\w+)", x)))

X_train_atsigncount = (X_train).apply(countAtSigns)
X_val_atsigncount = (X_val).apply(countAtSigns)
X_test_atsigncount = (X_test).apply(countAtSigns)

In [357]:
# Contar signos de porcentaje de un string

countPercentSigns = (lambda x: len(re.findall(r"%(\w+)", x)))

X_train_percentsigncount = (X_train).apply(countPercentSigns)
X_val_percentsigncount = (X_val).apply(countPercentSigns)
X_test_percentsigncount = (X_test).apply(countPercentSigns)

In [358]:
# Contar cuantos links (http) en un string

countLinks = (lambda x: x.count("http"))

X_train_linkcount = (X_train).apply(countLinks)
X_val_linkcount = (X_val).apply(countLinks)
X_test_linkcount = (X_test).apply(countLinks)

In [359]:
# Contar letras en un string

countLetters = (lambda x: sum(c.isalpha() for c in x))

X_train_lettercount = (X_train).apply(countLetters)
X_val_lettercount = (X_val).apply(countLetters)
X_test_lettercount = (X_test).apply(countLetters)